In [1]:
# !pip install pymupdf
# !pip install pdfminer.six
# 


In [8]:
import os
import re
import pandas as pd
import numpy as np
from lxml import etree
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve
import joblib

# === Paths ===
LABELS_CSV = "/kaggle/input/make-data-count-finding-data-references/train_labels.csv"
XML_DIR = "/kaggle/input/make-data-count-finding-data-references/train/XML/"
PDF_DIR = "/kaggle/input/make-data-count-finding-data-references/train/PDF/"
MODEL_PATH = "clf_model.joblib"
VECTORIZER_PATH = "vectorizer.joblib"

# === Load Labels ===
print("📥 Loading labels...")
labels_df = pd.read_csv(LABELS_CSV)

# === Utilities ===
def extract_text_from_xml(file_path):
    try:
        tree = etree.parse(file_path)
        return " ".join(tree.xpath('//text()'))
    except Exception:
        return ""

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as f:
            return f.read().decode("latin1", errors="ignore")
    except:
        return ""

def normalize_doi(doi):
    doi = doi.strip().lower()
    if doi.startswith("https://doi.org/"):
        return doi
    if doi.startswith("doi:"):
        doi = doi[4:]
    if doi.startswith("10."):
        return "https://doi.org/" + doi
    return doi

def extract_references(text):
    dois = re.findall(r'\b10\.\d{4,9}/[-._;()/:a-z0-9]+', text, flags=re.I)
    accessions = re.findall(r'\b(GSE\d+|E-[A-Z]+-\d+|PRJ[EDNA]\d+|PDB\s*\w+)\b', text, flags=re.I)
    references = set()
    for doi in dois:
        references.add(normalize_doi(doi))
    for acc in accessions:
        references.add(acc.replace(" ", "").upper())
    return references

# === Data Collection ===
print("🔍 Extracting training samples...")
contexts, labels = [], []

for _, row in labels_df.iterrows():
    article_id, dataset_id, label_type = row['article_id'], row['dataset_id'], row['type']
    xml_file = os.path.join(XML_DIR, article_id + ".xml")
    pdf_file = os.path.join(PDF_DIR, article_id + ".pdf")

    full_text = ""
    if os.path.exists(xml_file):
        full_text = extract_text_from_xml(xml_file)
    elif os.path.exists(pdf_file):
        full_text = extract_text_from_pdf(pdf_file)

    if not full_text:
        continue

    match = re.search(re.escape(dataset_id), full_text, flags=re.I)
    if match:
        snippet = full_text[max(match.start() - 300, 0):match.end() + 300].replace('\n', ' ')
    else:
        snippet = full_text[:300]

    contexts.append(snippet)
    labels.append(1 if label_type.lower() == "primary" else 0)

print(f"✅ Loaded {len(contexts)} samples. Label distribution: {Counter(labels)}")

# === TF-IDF Encoding with n-grams ===
print("🔠 Vectorizing text with TF-IDF...")
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 3))
X = vectorizer.fit_transform(contexts)
y = np.array(labels)

# === Train/Test Split ===
X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# === Model Training ===
clf = LogisticRegression(max_iter=1000, class_weight='balanced')
clf.fit(X_train, y_train)

# === Threshold Optimization ===
y_pred_proba = clf.predict_proba(X_val)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_val, y_pred_proba)
f1_scores = 2 * (precisions * recalls) / (precisions + recalls + 1e-6)
best_idx = np.argmax(f1_scores)

print("\n📊 Evaluation Report:")
print(classification_report(y_val, clf.predict(X_val)))
print(f"Best threshold: {thresholds[best_idx]:.2f}, F1: {f1_scores[best_idx]:.4f}")

# === Save Model & Vectorizer ===
joblib.dump(clf, MODEL_PATH)
joblib.dump(vectorizer, VECTORIZER_PATH)
print("💾 Model and vectorizer saved.")


📥 Loading labels...
🔍 Extracting training samples...
✅ Loaded 1028 samples. Label distribution: Counter({0: 758, 1: 270})
🔠 Vectorizing text with TF-IDF...

📊 Evaluation Report:
              precision    recall  f1-score   support

           0       0.94      0.95      0.95       152
           1       0.87      0.83      0.85        54

    accuracy                           0.92       206
   macro avg       0.90      0.89      0.90       206
weighted avg       0.92      0.92      0.92       206

Best threshold: 0.58, F1: 0.8600
💾 Model and vectorizer saved.


In [9]:
import os
import re
import pandas as pd
import joblib
from lxml import etree

# === Paths ===
TEST_XML_DIR = "/kaggle/input/make-data-count-finding-data-references/test/XML"
TEST_PDF_DIR = "/kaggle/input/make-data-count-finding-data-references/test/PDF"
MODEL_PATH = "clf_model.joblib"
VECTORIZER_PATH = "vectorizer.joblib"
SUBMISSION_FILE = "submission.csv"
DEBUG_LOG = "debug_log.csv"
BEST_THRESH = 0.58

# === Load Artifacts ===
clf = joblib.load(MODEL_PATH)
vectorizer = joblib.load(VECTORIZER_PATH)

# === Utilities ===
def extract_text_from_xml(file_path):
    try:
        tree = etree.parse(file_path)
        return " ".join(tree.xpath('//text()'))
    except:
        return ""

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, "rb") as f:
            return f.read().decode("latin1", errors="ignore")
    except:
        return ""

def extract_text(article_id):
    xml_path = os.path.join(TEST_XML_DIR, article_id + ".xml")
    pdf_path = os.path.join(TEST_PDF_DIR, article_id + ".pdf")
    if os.path.exists(xml_path):
        return extract_text_from_xml(xml_path)
    elif os.path.exists(pdf_path):
        return extract_text_from_pdf(pdf_path)
    return ""

def normalize_doi(doi):
    doi = doi.strip().lower()
    if doi.startswith("https://doi.org/"):
        return doi
    if doi.startswith("doi:"):
        doi = doi[4:]
    if doi.startswith("10."):
        return "https://doi.org/" + doi
    return doi

def extract_references(text):
    dois = re.findall(r'\b10\.\d{4,9}/[-._;()/:a-z0-9]+', text, flags=re.I)
    accessions = re.findall(r'\b(GSE\d+|E-[A-Z]+-\d+|PRJ[EDNA]\d+|PDB\s*\w+)\b', text, flags=re.I)
    references = set()
    for doi in dois:
        references.add(normalize_doi(doi))
    for acc in accessions:
        references.add(acc.replace(" ", "").upper())
    return references

# === Inference ===
print("🔍 Running inference on test set...")
submission_rows = []
debug_log = []

for fname in os.listdir(TEST_XML_DIR):
    if not fname.endswith(".xml"):
        continue
    article_id = fname.replace(".xml", "")
    text = extract_text(article_id)
    if not text:
        continue

    refs = extract_references(text)
    for dataset_id in refs:
        match = re.search(re.escape(dataset_id), text, flags=re.I)
        if match:
            snippet = text[max(match.start() - 300, 0):match.end() + 300].replace("\n", " ")
        else:
            snippet = text[:300]

        features = vectorizer.transform([snippet])
        prob = clf.predict_proba(features)[0, 1]
        citation_type = "Primary" if prob >= BEST_THRESH else "Secondary"

        # Save submission row
        submission_rows.append({
            "article_id": article_id,
            "dataset_id": dataset_id,
            "type": citation_type
        })

        # Save debug row
        debug_log.append({
            "article_id": article_id,
            "dataset_id": dataset_id,
            "probability": round(prob, 4),
            "predicted_type": citation_type,
            "snippet": snippet[:200].replace('\n', ' ')
        })

# === Save Files ===
submission_df = pd.DataFrame(submission_rows).drop_duplicates()
submission_df.insert(0, "row_id", range(len(submission_df)))
submission_df.to_csv(SUBMISSION_FILE, index=False)

debug_df = pd.DataFrame(debug_log)
debug_df.to_csv(DEBUG_LOG, index=False)

print("✅ submission.csv and debug_log.csv saved.")


🔍 Running inference on test set...
✅ submission.csv and debug_log.csv saved.


In [ ]:
# import os
# import re
# import pandas as pd
# import numpy as np
# from lxml import etree
# from scipy.sparse import hstack
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# import joblib

# # === Configuration ===
# TEST_XML_FOLDER = "/kaggle/input/make-data-count-finding-data-references/test/XML"  # Change to your test XML path
# OUTPUT_CSV = "submission.csv"
# MODEL_PATH = "/kaggle/working/clf_model.joblib"
# VECTORIZER_PATH = "/kaggle/working/vectorizer.joblib"
# BEST_THRESHOLD = 0.54  # from training

# # === Load Artifacts ===
# clf = joblib.load(MODEL_PATH)
# vectorizer = joblib.load(VECTORIZER_PATH)

# def extract_text_from_xml(file_path):
#     try:
#         tree = etree.parse(file_path)
#         return " ".join(tree.xpath('//text()'))
#     except Exception:
#         return ""

# def normalize_doi(doi):
#     doi = doi.strip().lower()
#     if doi.startswith("https://doi.org/"):
#         return doi
#     if doi.startswith("doi:"):
#         doi = doi[4:]
#     if doi.startswith("10."):
#         return "https://doi.org/" + doi
#     return doi

# def extract_references(text):
#     dois = re.findall(r'\b10\.\d{4,9}/[-._;()/:a-z0-9]+', text, flags=re.I)
#     accessions = re.findall(r'\b(GSE\d+|E-[A-Z]+-\d+|PRJ[EDNA]\d+|PDB\s*\w+)\b', text, flags=re.I)
#     references = set()

#     for doi in dois:
#         doi_full = normalize_doi(doi)
#         references.add(doi_full)

#     for acc in accessions:
#         acc_clean = acc.replace(" ", "").upper()
#         references.add(acc_clean)

#     return list(references)

# def keyword_feature(text):
#     keywords = ['this study', 'generated', 'deposited', 'we used', 'available at']
#     return int(any(k in text.lower() for k in keywords))

# # === Inference ===
# submission_rows = []

# for file_name in os.listdir(TEST_XML_FOLDER):
#     if not file_name.endswith(".xml"):
#         continue

#     article_id = file_name.replace(".xml", "")
#     xml_path = os.path.join(TEST_XML_FOLDER, file_name)
#     text = extract_text_from_xml(xml_path)
#     refs = extract_references(text)

#     for dataset_id in refs:
#         match = re.search(re.escape(dataset_id), text, flags=re.I)
#         snippet = text[max(match.start() - 100, 0):match.end() + 100] if match else text[:150]
#         snippet = snippet.replace('\n', ' ')

#         x_tfidf = vectorizer.transform([snippet])
#         x_kw = np.array([keyword_feature(snippet)]).reshape(-1, 1)
#         x_feat = hstack([x_tfidf, x_kw])

#         prob = clf.predict_proba(x_feat)[:, 1][0]
#         pred_type = "Primary" if prob >= BEST_THRESHOLD else "Secondary"

#         submission_rows.append({
#             "article_id": article_id,
#             "dataset_id": dataset_id,
#             "type": pred_type
#         })

# # === Save Submission ===
# submission_df = pd.DataFrame(submission_rows).drop_duplicates()
# submission_df.insert(0, "row_id", range(len(submission_df)))
# submission_df.to_csv(OUTPUT_CSV, index=False)

# print(f"✅ Submission file saved to: {OUTPUT_CSV}")


In [ ]:
# pip install pymupdf


In [5]:
# import os
# import re
# import pandas as pd
# import numpy as np
# import joblib
# from lxml import etree
# from sentence_transformers import SentenceTransformer

# # === Paths ===
# TEST_XML_DIR = "/kaggle/input/make-data-count-finding-data-references/test/XML"
# TEST_PDF_DIR = "/kaggle/input/make-data-count-finding-data-references/test/PDF"
# SUBMISSION_FILE = "submission.csv"
# MODEL_PATH = "clf_model.joblib"
# BERT_MODEL = "all-MiniLM-L6-v2"

# # === Load Model ===
# clf = joblib.load(MODEL_PATH)
# bert = SentenceTransformer(BERT_MODEL)

# # === Threshold ===
# BEST_THRESH = 0.56  # From training script

# # === Utility Functions ===
# def extract_text_from_xml(file_path):
#     try:
#         tree = etree.parse(file_path)
#         return " ".join(tree.xpath('//text()'))
#     except:
#         return ""

# def extract_text_from_pdf(pdf_path):
#     try:
#         with open(pdf_path, "rb") as f:
#             return f.read().decode("latin1", errors="ignore")
#     except:
#         return ""

# def extract_text(article_id):
#     xml_path = os.path.join(TEST_XML_DIR, article_id + ".xml")
#     pdf_path = os.path.join(TEST_PDF_DIR, article_id + ".pdf")
#     if os.path.exists(xml_path):
#         return extract_text_from_xml(xml_path)
#     elif os.path.exists(pdf_path):
#         return extract_text_from_pdf(pdf_path)
#     return ""

# def normalize_doi(doi):
#     doi = doi.strip().lower()
#     if doi.startswith("https://doi.org/"):
#         return doi
#     if doi.startswith("doi:"):
#         doi = doi[4:]
#     if doi.startswith("10."):
#         return "https://doi.org/" + doi
#     return doi

# def extract_references(text):
#     dois = re.findall(r'\b10\.\d{4,9}/[-._;()/:a-z0-9]+', text, flags=re.I)
#     accessions = re.findall(r'\b(GSE\d+|E-[A-Z]+-\d+|PRJ[EDNA]\d+|PDB\s*\w+)\b', text, flags=re.I)
#     references = set()
#     for doi in dois:
#         doi_full = normalize_doi(doi)
#         references.add(doi_full)
#     for acc in accessions:
#         acc_clean = acc.replace(" ", "").upper()
#         references.add(acc_clean)
#     return references

# # === Inference ===
# print("🔍 Running inference on test set...")
# submission_rows = []

# for fname in os.listdir(TEST_XML_DIR):
#     if not fname.endswith(".xml"):
#         continue
#     article_id = fname.replace(".xml", "")
#     text = extract_text(article_id)
#     if not text:
#         continue

#     refs = extract_references(text)
#     for dataset_id in refs:
#         match = re.search(re.escape(dataset_id), text, flags=re.I)
#         if match:
#             snippet = text[max(match.start() - 100, 0):match.end() + 100].replace("\n", " ")
#         else:
#             snippet = text[:150]

#         embedding = bert.encode([snippet])
#         prob = clf.predict_proba(embedding)[0, 1]
#         citation_type = "Primary" if prob >= BEST_THRESH else "Secondary"

#         submission_rows.append({
#             "article_id": article_id,
#             "dataset_id": dataset_id,
#             "type": citation_type
#         })

# # === Save Submission ===
# submission_df = pd.DataFrame(submission_rows).drop_duplicates()
# submission_df.insert(0, "row_id", range(len(submission_df)))
# submission_df.to_csv(SUBMISSION_FILE, index=False)
# print("✅ Submission file created as submission.csv")


🔍 Running inference on test set...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Submission file created as submission.csv
